## Import all Required library

In [1]:
import time
import pandas as pd
import transformers
import numpy as np
from transformers import AutoTokenizer,AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_metric
from datasets import Dataset
from utils import read_data_as_sentence,map_labels_in_dataframe,tokenize_and_align_labels,get_label_mapping,get_labels_from_map,load_srl_model,load_dataset,compute_metrics

## Preprocessing data

Read data files and save them as dataframe.\
\
Use `read_data_as_sentence` function from `utils.py` to read files as Dataframe.\

`read_data_as_sentence` need two input:
1. path of conllu file.
2. path for save datarame

In [2]:
train_data = read_data_as_sentence('data/en_ewt-up-train.conllu', 'data/en_ewt-up-train.preprocessed.csv')
dev_data = read_data_as_sentence('data/en_ewt-up-dev.conllu', 'data/en_ewt-up-dev.preprocessed.csv')
test_data = read_data_as_sentence('data/en_ewt-up-test.conllu', 'data/en_ewt-up-test.preprocessed.csv')

Show 20 input form and their gold list in test data

In [3]:
for i in range(20):
    print(f"{test_data.input_form[i]}\t:\t{test_data.argument[i]}")

['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?', '[SEP]', 'morph']	:	['_', '_', 'ARG1', '_', '_', 'ARG2', '_', None, '_']
['What', 'if', 'Google', 'expanded', 'on', 'its', 'search', '-', 'engine', '(', 'and', 'now', 'e-mail', ')', 'wares', 'into', 'a', 'full', '-', 'fledged', 'operating', 'system', '?', '[SEP]', 'expand']	:	['_', '_', 'ARG0', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'ARG1', '_', '_', '_', '_', '_', '_', 'ARG4', '_', None, '_']
['(', 'And', ',', 'by', 'the', 'way', ',', 'is', 'anybody', 'else', 'just', 'a', 'little', 'nostalgic', 'for', 'the', 'days', 'when', 'that', 'was', 'a', 'good', 'thing', '?', ')', '[SEP]', 'way']	:	['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', None, '_']
['(', 'And', ',', 'by', 'the', 'way', ',', 'is', 'anybody', 'else', 'just', 'a', 'little', 'nostalgic', 'for', 'the', 'days', 'when', 'that', 'was', 'a', 'good', 'thing', '?', ')', '[SEP]', 

Head of test data after process

In [4]:
test_data.head()

,input_form,argument
0,"[What, if, Google, Morphed, Into, GoogleOS, ?,...","[_, _, ARG1, _, _, ARG2, _, None, _]"
1,"[What, if, Google, expanded, on, its, search, ...","[_, _, ARG0, _, _, _, _, _, _, _, _, _, _, _, ..."
2,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."
3,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, ARGM-DIS, _, _, ARG1, _, _, _,..."
4,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."


In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4799 entries, 0 to 4798
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   input_form  4799 non-null   object
 1   argument    4799 non-null   object
dtypes: object(2)
memory usage: 75.1+ KB


## Importing the model and tokenizer

In [6]:
model_checkpoint = "distilbert-base-uncased"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Checking the sentence representation

In [8]:
example = test_data['input_form'][1]
print(example)

['What', 'if', 'Google', 'expanded', 'on', 'its', 'search', '-', 'engine', '(', 'and', 'now', 'e-mail', ')', 'wares', 'into', 'a', 'full', '-', 'fledged', 'operating', 'system', '?', '[SEP]', 'expand']


The sentence contains the [SEP] special token followed by the predicate. Therefore, the parameter `add_special_tokens` is set to True so that the index is converted to 102 accordingly and is not treated as another word. \
In addition, the sentence is already split into tokens, to the parameter `is_split_into_words` is also set to True

In [9]:
tokenizer(example,add_special_tokens=True,is_split_into_words=True)

{'input_ids': [101, 2054, 2065, 8224, 4423, 2006, 2049, 3945, 1011, 3194, 1006, 1998, 2085, 1041, 1011, 5653, 1007, 16283, 2015, 2046, 1037, 2440, 1011, 26712, 4082, 2291, 1029, 102, 7818, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
tokenized_input = tokenizer(example,add_special_tokens=True,is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'what', 'if', 'google', 'expanded', 'on', 'its', 'search', '-', 'engine', '(', 'and', 'now', 'e', '-', 'mail', ')', 'ware', '##s', 'into', 'a', 'full', '-', 'fledged', 'operating', 'system', '?', '[SEP]', 'expand', '[SEP]']


## Tokenizing and preparing input for the model

Getting the mapping of all possible arguments across all datasets to a numerical value with the `get_label_mapping` function.\
None value stays as None to be mapped to the special token in the model.

In [11]:
label_map = get_label_mapping(train_data, test_data, dev_data)

In [12]:
print(label_map)

{'_': 0, 'ARG0': 1, 'ARG1': 2, 'ARG1-DSP': 3, 'ARG2': 4, 'ARG3': 5, 'ARG4': 6, 'ARG5': 7, 'ARGA': 8, 'ARGM-ADJ': 9, 'ARGM-ADV': 10, 'ARGM-CAU': 11, 'ARGM-COM': 12, 'ARGM-CXN': 13, 'ARGM-DIR': 14, 'ARGM-DIS': 15, 'ARGM-EXT': 16, 'ARGM-GOL': 17, 'ARGM-LOC': 18, 'ARGM-LVB': 19, 'ARGM-MNR': 20, 'ARGM-MOD': 21, 'ARGM-NEG': 22, 'ARGM-PRD': 23, 'ARGM-PRP': 24, 'ARGM-PRR': 25, 'ARGM-REC': 26, 'ARGM-TMP': 27, 'C-ARG0': 28, 'C-ARG1': 29, 'C-ARG1-DSP': 30, 'C-ARG2': 31, 'C-ARG3': 32, 'C-ARG4': 33, 'C-ARGM-ADV': 34, 'C-ARGM-COM': 35, 'C-ARGM-CXN': 36, 'C-ARGM-DIR': 37, 'C-ARGM-EXT': 38, 'C-ARGM-GOL': 39, 'C-ARGM-LOC': 40, 'C-ARGM-MNR': 41, 'C-ARGM-PRP': 42, 'C-ARGM-PRR': 43, 'C-ARGM-TMP': 44, 'C-V': 45, 'R-ARG0': 46, 'R-ARG1': 47, 'R-ARG2': 48, 'R-ARG3': 49, 'R-ARG4': 50, 'R-ARGM-ADJ': 51, 'R-ARGM-ADV': 52, 'R-ARGM-CAU': 53, 'R-ARGM-COM': 54, 'R-ARGM-DIR': 55, 'R-ARGM-GOL': 56, 'R-ARGM-LOC': 57, 'R-ARGM-MNR': 58, 'R-ARGM-TMP': 59, None: None}


Converting the labels in the df to numerical values for the language model with `map_labels_in_dataframe` function. The label_map dictionary from the function above is needed to map the arguments to their value.\
Add a new column to the df matching the arguments to label numbers. 0 stands for '_' (no argument) and the rest of the arguments are alphabetically ordered. \
*None* label will be mapped to the *[SEP]* token.


In [13]:
train_data = map_labels_in_dataframe(train_data,label_map)
dev_data = map_labels_in_dataframe(dev_data,label_map)
test_data = map_labels_in_dataframe(test_data,label_map)

Checking the head to confirm the labels were correctly converted:

In [14]:
test_data.head()

,input_form,argument,mapped_labels
0,"[What, if, Google, Morphed, Into, GoogleOS, ?,...","[_, _, ARG1, _, _, ARG2, _, None, _]","[0, 0, 2, 0, 0, 4, 0, None, 0]"
1,"[What, if, Google, expanded, on, its, search, ...","[_, _, ARG0, _, _, _, _, _, _, _, _, _, _, _, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, ..."
2,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, ARGM-DIS, _, _, ARG1, _, _, _,...","[0, 0, 0, 0, 0, 15, 0, 0, 2, 0, 0, 0, 0, 4, 0,..."
4,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


Use `tokenize_and_align_labels` function to tokenize train, test, and dev dataframe. Padding is applied to make sure all input is the same length for the model.

In [15]:
tokenized_test = tokenize_and_align_labels(tokenizer, test_data, label_all_tokens=True)
tokenized_train = tokenize_and_align_labels(tokenizer, train_data, label_all_tokens=True)
tokenized_dev = tokenize_and_align_labels(tokenizer, dev_data, label_all_tokens=True)

The input for the model has the corresponding special token [CLS] followed by the tokenized sentence, the special token [SEP], the predicate and the final [SEP] token. \
The numerical labels to be fed to the model correspond to the tokenized sentence.\
The input is padded so that every vector is of the same length, including the labels and the attention mask.

In [16]:
print(tokenizer.convert_ids_to_tokens(tokenized_test["input_ids"][0]))
print(tokenized_test["attention_mask"][0])
print(tokenized_test["input_ids"][0])
print(tokenized_test["labels"][0])

['[CLS]', 'what', 'if', 'google', 'mor', '##ph', '##ed', 'into', 'google', '##os', '?', '[SEP]', 'mor', '##ph', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Confirming all the tokens contain a label and the attention mask also matches the length of the input

In [17]:
print(len(tokenized_test["input_ids"][0]),len(tokenized_test["labels"][0]),len(tokenized_test["attention_mask"][0]))

96 96 96


Converting the tokenized data to datasets format with the function `load_dataset`

In [18]:
dataset_train = load_dataset(tokenized_train)
dataset_dev = load_dataset(tokenized_dev)
dataset_test = load_dataset(tokenized_test)

## Fine-tuning the model

Reducing the size of the dataset for a mini test

In [27]:
small_train_dataset = dataset_train.shuffle(seed=42).select(range(1000))
small_eval_dataset = dataset_dev.shuffle(seed=42).select(range(1000))
small_test_dataset = dataset_test.shuffle(seed=42).select(range(1000))

Getting the labels that will be predicted by the model with the `get_labels_from_map`function

In [20]:
label_list = get_labels_from_map(label_map)

Loading the model for semantic role labelling task with function `load_srl_model` to get the model, its name and the arguments necessary for training. \
The model selected is **distilbert-base-uncased**

In [21]:
model, model_name, args = load_srl_model(model_checkpoint, label_list)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
print(model_name)

distilbert-base-uncased


Load the seqeval metric to compute the metrics from the predictions

In [23]:
metric = load_metric("seqeval")

/var/folders/c3/6b53cyc56hdcsrh90312r_1w0000gn/T/ipykernel_6368/152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


metric.compute(predictions=[label_list], references=[label_list])

Passing the arguments along with the datasets to the `trainer` function to fine-tune the model for semantic role labelling with `trainer.train()`

In [24]:
trainer = Trainer(
        model,
        args,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=lambda p: compute_metrics(*p, label_list, metric)
    )
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.143463,0.000000,0.000000,0.000000,0.910457
2,No log,0.133497,0.000000,0.000000,0.000000,0.910457
3,No log,0.130109,0.000000,0.000000,0.000000,0.910457


/Users/arianabritez/anaconda3/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ARGM-MNR seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/arianabritez/anaconda3/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: _ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/arianabritez/anaconda3/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ARG1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/arianabritez/anaconda3/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ARGM-GOL seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/arianabritez/anaconda3/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ARG0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE t

TrainOutput(global_step=189, training_loss=0.11271812802269346, metrics={'train_runtime': 130.4714, 'train_samples_per_second': 22.994, 'train_steps_per_second': 1.449, 'total_flos': 137942701920000.0, 'train_loss': 0.11271812802269346, 'epoch': 3.0})

Evaluate a model fine-tuned for semantic role labelling with `trainer.evaluate()`

In [25]:
trainer.evaluate()

{'eval_loss': 0.13010868430137634,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_accuracy': 0.910457081009075,
 'eval_runtime': 5.9225,
 'eval_samples_per_second': 168.849,
 'eval_steps_per_second': 10.637,
 'epoch': 3.0}

After training is finished, the precision/recall/f1 for each category can be computed. \
The same function `compute_metrics` is applied on the result of the predict method.

In [30]:
predictions, labels, _ = trainer.predict(small_eval_dataset)
results = compute_metrics(predictions, labels, label_list, metric)
results

{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.910457081009075}